In [1]:
import glob
import cv2
from PIL import Image,ImageDraw
import numpy as np
import re,json,os
from shapely.geometry import LineString, Point
from numpy.lib.stride_tricks import as_strided
from pathlib import Path

### TO DO:
#### Data generation
- [x] replace corner boxes by convex hulls coordinates - suggested scipy.spatial.ConvexHull and save it to dict along with image (see cards dicitionary - variable further down )
- [x] download background textures
- [x] investigate 'data/cards_jpeg/10c', 'data/cards_jpeg/Ks', 'data/cards_jpeg/10h'

- [ ]  create augmentation pipeline: perspective transformations, change in lightning. Show few examples to verify correctness. (remember to augment bboxes along with images) - suggested imgaug
- [ ]  figure out a way to automatically position cards on an image (e.g. one cards is nearly all covered by another). Choose randomly background texture but remember it should be more or less uniquely distributed 
- [ ]  create data, with images consisting with N - number of distinct cards on an image
#### Model
to be continued

In [2]:
files = glob.glob("data/cards_jpeg/*.jpeg")
files = [f.rstrip(".jpeg") for f in files]

In [3]:
def imshow(a):
    a = a.clip(0, 255).astype('uint8')
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(a))

In [4]:
class Card:
    def __init__(self,name):
        self.name = name
        self.image = None
        self.card_polygon = None
        self.label_polygons = []
        
    def set_image(self,image):
        self.image = image
        
    def set_card_polygon(self,card_polygon):
        self.card_polygon = card_polygon
        
    def set_label_polygons(self,label_polygons):
        self.label_polygons = label_polygons
    
    def add_label_polygon(self,label_polygon):
        self.label_polygons.append(label_polygon)
    
        
    def save(self,dir_name='./interim_data'):
        Path(dir_name).mkdir(parents=True, exist_ok=True)
        template = {"filename":self.name,
                   "card_polygon":self.card_polygon,
                   "label_polygons": self.label_polygons}
        
        json_name = os.path.join(dir_name,self.name+'.json')
        with open(json_name,'w') as fp:
            json.dump(template,fp)
        print(f"{json_name} saved correctly!")
        
        image_name = os.path.join(dir_name,self.name+'.png')
        cv2.imwrite(image_name,self.image)
        print(f"{image_name} saved correctly!")
    
    
    

In [5]:
def rolling_window(a, window_size):
    shape = (a.shape[0] - window_size + 1, window_size) + a.shape[1:]
    strides = (a.strides[0],) + a.strides
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [6]:
def get_longest_lines(card):
    """return lines [x1,y1,x2,y2]"""
    pairs_of_points = rolling_window(card.card_polygon,2)
    points1 ,points2= pairs_of_points[:,0,:],pairs_of_points[:,1,:]
    distances = np.apply_along_axis(np.linalg.norm, 1, points1-points2)
    return pairs_of_points[distances.argsort()][-4:].reshape(-1,4)

In [7]:
def extend_lines(lines,height,width):
    """extends line segments to borders of image"""
    extended_lines = []
    for line in lines:
        x1,y1,x2,y2 = line
        numerator = (y2 - y1) 
        denumerator = (x2 - x1)
        if denumerator == 0:
            a = numerator / 0.001
        else:
            a = numerator / denumerator
        b = y1 - a * x1
        
        if abs(a) > 70:
            extended_lines.append([(-b)/a,0,(height-b)/a,height])
        else:
            extended_lines.append([0,b,width,a*width +b])
    extended_lines = np.array(extended_lines).astype(np.int32)
    return extended_lines
            

In [8]:
def get_intersection_points(lines):
    lines = np.vstack((lines,np.expand_dims(lines[0],axis=0)))
    pairs_of_lines = rolling_window(lines, 2)
    intersections = []
    for pair in pairs_of_lines:
        A,B,C,D = pair.reshape(4,2)
        line1 = LineString([A, B])
        line2 = LineString([C, D])

        int_pt = line1.intersection(line2)
        point_of_intersection = int_pt.x, int_pt.y
        intersections.append(point_of_intersection)
    return np.array(intersections).astype(np.float32)

In [9]:
def get_bbox(card):
    lines = get_longest_lines(card)
    im = card.image.copy()
    height,width = card.image.shape[:2]
    extended_lines = extend_lines(lines,height,width) 
    intersection_points = get_intersection_points(extended_lines[[0,2,1,3]]) #gora prawo dol lewo
    return extended_lines,intersection_points

In [10]:
def get_minAreaRect(intersection_points):
    x_s,y_s = intersection_points[:,0],intersection_points[:,1]
    minx,maxx = x_s[x_s.argsort()][[0,3]]
    miny,maxy = y_s[y_s.argsort()][[0,3]]
    return np.array([[minx,miny],[maxx,miny],[maxx,maxy],[minx,maxy]])

In [11]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    (tl, tr, br, bl) = pts
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped,M

In [12]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [13]:
def apply_mask(image):
    im = card.image.copy()
    im = cv2.cvtColor(im,cv2.COLOR_BGR2BGRA).astype(np.uint8)
    img = Image.new('L', im.shape[:2][::-1], 0)
    ImageDraw.Draw(img).polygon([tuple(x) for x in card.card_polygon], outline=255, fill=255)
    resized_mask = cv2.resize(np.array(img),np.array(im.shape[:2][::-1])-26).astype(np.uint8)
    resized_mask = np.pad(resized_mask,13,'constant',constant_values=0)
    im[:,:,3] = np.array(resized_mask)
    return im

In [14]:
def rotate_polygon(polygon,M):
    pts = np.array(polygon)
    pts = np.hstack((pts,np.ones((pts.shape[0],1))))
    pts = np.float32(np.dot(pts,M.T))[:,:2]
    return numpy_to_list(pts)

In [15]:
def numpy_to_list(array):
    lst2 = []
    for i in array:
        lst = []
        for k in i:
            lst.append(float(k))
        lst2.append(lst)
    return lst2

In [16]:
for file in files:
    card = Card(file.split("/")[-1])
    annotations = json.load(open(file+".json",'r'))['shapes']
    for shape in annotations:
        if shape['label'] == 'card':
            card.set_card_polygon(np.array(shape['points']).astype(np.int32))
        else:
            card.add_label_polygon(shape['points'])
    image = cv2.imread(file+".jpeg")
    card.set_image(image)
    img = apply_mask(card.image.copy())
    extended_lines,intersection_points = get_bbox(card)
    min_area_rect = get_minAreaRect(intersection_points)
    img,M = four_point_transform(img,order_points(intersection_points))
    card.set_card_polygon(rotate_polygon(card.card_polygon,M))
    card.set_label_polygons([rotate_polygon(polygon,M) for polygon in card.label_polygons])
    card.set_image(img)

    card.save()

./interim_data/Ah.json saved correctly!
./interim_data/Ah.png saved correctly!
./interim_data/4h.json saved correctly!
./interim_data/4h.png saved correctly!
./interim_data/5h.json saved correctly!
./interim_data/5h.png saved correctly!
./interim_data/8s.json saved correctly!
./interim_data/8s.png saved correctly!
./interim_data/Jd.json saved correctly!
./interim_data/Jd.png saved correctly!
./interim_data/8h.json saved correctly!
./interim_data/8h.png saved correctly!
./interim_data/7s.json saved correctly!
./interim_data/7s.png saved correctly!
./interim_data/9c.json saved correctly!
./interim_data/9c.png saved correctly!
./interim_data/2d.json saved correctly!
./interim_data/2d.png saved correctly!
./interim_data/6s.json saved correctly!
./interim_data/6s.png saved correctly!
./interim_data/3s.json saved correctly!
./interim_data/3s.png saved correctly!
./interim_data/Jc.json saved correctly!
./interim_data/Jc.png saved correctly!
./interim_data/6h.json saved correctly!
./interim_da